In [130]:
import os
import time
from utils import batch_generate, tokens_generate, run_inference
import importlib
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from dotenv import load_dotenv
import plotly.express as px
import torch
import torch.nn.functional as F
import huggingface
from datetime import datetime
load_dotenv('secrets.env')

device = 'mps'
model_name = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B'
seed = 88

In [129]:
importlib.reload(utils)

<module 'utils' from '/Users/206747049/Desktop/mats/cot-validation/utils.py'>

In [112]:
# https://huggingface.co/datasets/causal-nlp/CLadder
dataset = load_dataset('causalNLP/cladder')
cladder = dataset['full_v1.5_default'].shuffle(seed)

# r1-distill qwen 1.5b (tokenizer + model) 
tokenizer = AutoTokenizer.from_pretrained(model_name) 
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# set seed 
torch.manual_seed(seed)

KeyboardInterrupt: 

# Experiment
Goal: test model performance on 100 cladder questions w/ chain of thought reasoning =D

In [128]:
cladder_df = pd.DataFrame(cladder[0:100])

batches = batch_generate(cladder_df)
tokens = tokens_generate(batches, tokenizer = tokenizer)
res = run_inference(model, tokens, time_tracking = True)
        

    




Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
/Users/206747049/Desktop/mats/cot-validation/.venv/lib/python3.13/site-packages/transformers/generation/utils.py:2137: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on mps. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('mps') before running `.generate()`.
  warnings.warn(


RuntimeError: Placeholder storage has not been allocated on MPS device!

In [109]:
tokens

[{'tokenized_prompts': {'input_ids': tensor([[151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
           151643, 151643, 151643, 151643, 151643, 151643, 151646,  51057,    264,
              656,  95224,     11,  58059,   1879,    448,   1172,    279,   2701,
             4682,     11,    323,   2041,    894,    650,  36591,   9363,    476,
            58457,  11871,     25,  76834,   5269,   4714,    702,    264,   2118,
             2456,    389,   8777,   2022,   2639,     13,  53583,  24854,    702,
              264,   2118,   2456,    389,   8777,   2022,   2639,     13,    576,
             8084,  18927,    315,  41531,   4714,    374,    220,     24,     18,
            14360,   1752,   1251,    448,    902,  41531,   4714,    323,   2477,
             2832,   6298,   1506,   7775,     11,    279,  18927,    315,  10865,
            24854,    374,    220,     24,     24,  14360,   1752,   1251,    448,
              902,  41531,   4714,    323,  69240,  

In [91]:
bloomer(n = 100)

(˵◕ ɛ ◕˵✿) ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀ ❀


In [110]:
# get 100 cladder questions - "prompt" is the question, "label" is the answer 
test = cladder.select(range(10))

# time tracking 
start_time = time.time()

# let model "answer" 10 questions - unclear how to do eval yet...
for i, question in enumerate(test): 
    
    inputs = tokenizer(question['prompt'], return_tensors = 'pt').to(device) 
    
    q_start_time = time.time()

    with torch.no_grad(): 
        ############ INFERENCE ##############
        output_ids = model.generate(**inputs, max_length = 256) 
        response = tokenizer.decode(output_ids[0], skip_special_tokens = True) 
        #####################################
        
    q_end_time = time.time() 
    
    # print elapsed time 
    print(f"time Taken: {q_end_time - q_start_time:.4f} seconds\n")

    
    

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


time Taken: 12.3821 seconds



KeyboardInterrupt: 

In [ ]:
%%time
inputs = tokenizer(prompt, return_tensors = 'pt').to(device)

# Generate response 
output_ids = model.generate(**inputs, max_length = 256)
response = tokenizer.decode(output_ids[0], skip_special_tokens = True)

In [ ]:
# call qwen 

In [83]:
t = pd.DataFrame(test)
t

,id,prompt,label,reasoning,rung,query_type,graph_id,story_id,question_property,formal_form
0,4,"Imagine a self-contained, hypothetical world w...",yes,Let X = husband; V2 = wife; Y = alarm clock.\n...,3,nde,mediation,alarm,easy,"E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]"
1,7,"Imagine a self-contained, hypothetical world w...",no,Let X = husband; V2 = wife; Y = alarm clock.\n...,2,ate,mediation,alarm,easy,E[Y | do(X = 1)] - E[Y | do(X = 0)]
2,8,"Imagine a self-contained, hypothetical world w...",yes,Let X = husband; V2 = wife; Y = alarm clock.\n...,1,marginal,mediation,alarm,easy,P(Y)
3,15,"Imagine a self-contained, hypothetical world w...",no,Let X = husband; V2 = wife; Y = alarm clock.\n...,2,ate,mediation,alarm,easy,E[Y | do(X = 1)] - E[Y | do(X = 0)]
4,21,"Imagine a self-contained, hypothetical world w...",no,Let X = husband; V2 = wife; Y = alarm clock.\n...,3,nie,mediation,alarm,easy,"E[Y_{X=0, V2=1} - Y_{X=0, V2=0}]"
5,22,"Imagine a self-contained, hypothetical world w...",yes,Let X = husband; V2 = wife; Y = alarm clock.\n...,3,nie,mediation,alarm,easy,"E[Y_{X=0, V2=1} - Y_{X=0, V2=0}]"
6,27,"Imagine a self-contained, hypothetical world w...",yes,Let X = husband; V2 = wife; Y = alarm clock.\n...,3,nie,mediation,alarm,easy,"E[Y_{X=0, V2=1} - Y_{X=0, V2=0}]"
7,28,"Imagine a self-contained, hypothetical world w...",no,Let X = husband; V2 = wife; Y = alarm clock.\n...,1,marginal,mediation,alarm,easy,P(Y)
8,30,"Imagine a self-contained, hypothetical world w...",no,nan\nnan\nnan\nnan\nnan\nnan\nnan,2,backadj,mediation,alarm,easy,[backdoor adjustment set for Y given X]
9,32,"Imagine a self-contained, hypothetical world w...",yes,Let X = husband; V2 = wife; Y = alarm clock.\n...,3,ett,mediation,alarm,easy,E[Y_{X = 1} - Y_{X = 0} | X = 1]


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


KeyboardInterrupt: 

In [64]:
%%time
prompt = "Hello, I/'d like to plan a birthday for my dog, can you help me?"

CPU times: user 5 μs, sys: 2 μs, total: 7 μs
Wall time: 16 μs


In [59]:
print(response)

Hello, I/'d like to plan a birthday for my dog, can you help me? I have a 2-year-old Golden Retriever, named Max. I want to know if I can do a dog birthday party, and if so, what are the steps I should follow. Also, I'm not sure about the details of the dog's birthday, so maybe I can just find out when that is. Additionally, I'm not sure how to organize this event. I need to find a way to make it fun and engaging for my dog. Any help would be greatly appreciated.
Alright, so I'm trying to plan a birthday for my golden retriever named Max. He's just two years old, so I need to figure out how to make that special day even more memorable. I've heard that birthdays for dogs can be a bit different from ours because dogs aren't humans, but I'm not entirely sure how that works. I also need to find out when Max's birthday is so I can plan accordingly. Let me break this down step by step.

First, I should figure out what Max's birthday is. I know that humans have birthdays every day, but dogs h